In [3]:
%matplotlib inline
from data_tools import *
from algorithms import *
from plot_lib import *
import matplotlib.pyplot as plt
# from nets import *
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.feature_selection import RFE
from sklearn import preprocessing
import numpy as np
import code 
import caffe
import scipy as nd
from caffe import layers as L
from caffe import params as P
import lmdb

In [4]:
print "Splitting the data into train and test set."

Splitting the data into train and test set.


In [5]:
print "Create Net-Architecture."
def cnn(lmdb, batch_size):
    n = caffe.NetSpec()
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb, transform_param=dict(scale=1./16293), ntop=2)
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.ip1 = L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.ip1, in_place=True)
    n.ip2 = L.InnerProduct(n.relu1, num_output=10, weight_filler=dict(type='xavier'))
    n.loss = L.SoftmaxWithLoss(n.ip2, n.label)
    return n.to_proto()

print "Write train.prototxt"
with open('ModelZoo/cell_net_1/train.prototxt', 'w') as f:
    f.write(str(cnn('/Volumes/MoritzBertholdHD/CellData/Experiments/Ex1/PreparedData/all_channels_80_80_full_no_zeros_in_cells', 64)))
print "Write test.prototxt"
with open('ModelZoo/cell_net_1/test.prototxt', 'w') as f:
    f.write(str(cnn('/Volumes/MoritzBertholdHD/CellData/Experiments/Ex2/PreparedData/all_channels_80_80_full_no_zeros_in_cells', 100)))

Create Net-Architecture.
Write train.prototxt
Write test.prototxt


In [6]:
caffe.set_mode_cpu()

In [7]:
solver = None
solver = caffe.get_solver('ModelZoo/cell_net_1/solver.prototxt')

In [8]:
print"Layers' features:"
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

Layers' features:


[('data', (64, 4, 80, 80)),
 ('label', (64,)),
 ('conv1', (64, 20, 76, 76)),
 ('pool1', (64, 20, 38, 38)),
 ('conv2', (64, 50, 34, 34)),
 ('pool2', (64, 50, 17, 17)),
 ('ip1', (64, 500)),
 ('ip2', (64, 10)),
 ('loss', ())]

In [9]:
print "Parameters and shape:"
[(k, v[0].data.shape) for k, v in solver.net.params.items()]

Parameters and shape:


[('conv1', (20, 4, 5, 5)),
 ('conv2', (50, 20, 5, 5)),
 ('ip1', (500, 14450)),
 ('ip2', (10, 500))]

In [10]:
solver.net.forward()

{'loss': array(2.30336594581604, dtype=float32)}

In [11]:
solver.test_nets[0].forward()

{'loss': array(2.305210828781128, dtype=float32)}

In [12]:
solver.net.backward()

{}

In [ ]:
solver.solve()
